In [1]:
import mlflow
import mlflow.pyfunc
import mlflow.pytorch
from  mlflow.tracking import MlflowClient
import os
import numpy as np
import torch
from torchvision import datasets, transforms
import pandas as pd

kwargs = {}
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=128, shuffle=True, **kwargs)

In [16]:
experiment_name = "pytorch_exp_1"
tracking_uri = os.environ.get("TRACKING_URL")
client = MlflowClient(tracking_uri=tracking_uri)
mlflow.set_tracking_uri(tracking_uri)
experiments = client.list_experiments()

experiment_names = []
for exp in experiments:
    experiment_names.append(exp.name)
if experiment_name not in experiment_names:
    try:
        mlflow.create_experiment(experiment_name)
    except:
        pass
mlflow.set_experiment(experiment_name)

In [128]:
run_id = "e41989fbd5334b7591a8fe856c7bc907"
with mlflow.start_run(run_id=run_id):
    loaded_model = mlflow.pyfunc.load_model(mlflow.get_artifact_uri("pytorch-model"))

    # Extract a few examples from the test dataset to evaulate on
    eval_data, eval_labels = next(iter(test_loader))
    print(eval_data.dtype)
    # Make a few predictions
    predictions = loaded_model.predict(eval_data).data.max(1)[1]
    template = 'Sample {} : Ground truth is "{}", model prediction is "{}"'
    print("\nSample predictions")
    for index in range(5):
        print(template.format(index, eval_labels[index], predictions[index]))


Pytorch model initialized
torch.float32
Predicting 128 samples

Sample predictions
Sample 0 : Ground truth is "3", model prediction is "3"
Sample 1 : Ground truth is "1", model prediction is "1"
Sample 2 : Ground truth is "3", model prediction is "3"
Sample 3 : Ground truth is "2", model prediction is "2"
Sample 4 : Ground truth is "1", model prediction is "8"


In [35]:
import base64
import cv2

In [5]:
from PIL import Image
import base64
import io
import numpy as np
import torch

In [36]:
filename = "/home/ubuntu/mnist_png/testing/0/10.png"
cvdata = cv2.imread(filename)

In [10]:
import os
from glob import glob
_path = "/home/ubuntu/mnist_png/testing/"
file_list = [y for x in os.walk(_path) for y in glob(os.path.join(x[0], '*.png'))]

In [149]:
correct_labels = []
for _file in file_list:
    correct_labels.append(_file.split("/")[-2])

In [150]:
test_size = min(len(file_list),100)
file_list = file_list[:test_size]
correct_labels = correct_labels[:test_size]
nparray_list = []
for _file in file_list:
    with open(_file, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    base64_decoded = base64.b64decode(encoded_string)
    image = Image.open(io.BytesIO(base64_decoded))
    image_np = np.array(image,dtype=np.uint8)
    image_np = np.expand_dims(image_np,axis=0)
    nparray_list.append(image_np)
nparray = np.stack(nparray_list,axis=0).astype(np.float32)
torch_tensor = torch.from_numpy(nparray)

In [ ]:
run_id = "8ae8c49e85f24f98a6b33a4e52c9f7bb"
with mlflow.start_run(run_id=run_id):
    loaded_model = mlflow.pyfunc.load_model(mlflow.get_artifact_uri("pytorch-model"))

    # Make a few predictions
    predictions = loaded_model.predict(torch_tensor).data.max(1)[1]
    template = 'Sample {} : Ground truth is "{}", model prediction is "{}"'
    print("\nSample predictions")
    for index in range(100):
        print(template.format(index,correct_labels[index],predictions[index]))


In [12]:
test_size = min(len(file_list),100)
file_list = file_list[:test_size]
encoded_string_list = []
# correct_labels = correct_labels[:test_size]
nparray_list = []
for _file in file_list:
    with open(_file, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    encoded_string_list.append(encoded_string)
input_df = pd.DataFrame(encoded_string_list,columns=["images"])

In [39]:
run_id = "9fb2052dc0da45f5b888cfa645666ef2"
with mlflow.start_run(run_id=run_id):
    loaded_model = mlflow.pyfunc.load_model(mlflow.get_artifact_uri("pytorch-model"))

    # Make a few predictions
    predictions = loaded_model.predict(input_df)
#     template = 'Sample {} : Ground truth is "{}", model prediction is "{}"'
#     print("\nSample predictions")
#     for index in range(100):
#         print(template.format(index,correct_labels[index],predictions[index]))
predictions

Pytorch model initialized
Predicting 100 samples


array([5, 4, 3, 3, 5, 5, 3, 5, 5, 5, 7, 5, 5, 5, 0, 5, 5, 5, 3, 9, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 9, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 7, 5, 5,
       8, 5, 5, 5, 5, 5, 5, 5, 6, 8, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5,
       5, 5, 0, 5, 5, 3, 5, 5, 2, 5, 5, 5, 5, 9, 5, 8, 5, 8, 3, 5, 5, 5,
       5, 8, 1, 5, 3, 5, 5, 5, 3, 5, 6, 5])

In [28]:
conda_env["dependencies"].append("cpuonly")

In [29]:
conda_env

{'name': 'mlflow-env',
 'channels': ['defaults', 'conda-forge', 'pytorch'],
 'dependencies': ['python=3.7.7',
  'pytorch=1.7.1',
  'torchvision=0.8.2',
  'pip',
  {'pip': ['mlflow', 'cloudpickle==1.4.1']},
  'cpuonly']}

In [9]:
!pip freeze | grep PIL

In [43]:
import requests
url = "https://awfqvt3.azure.babyrocket.net/invocations"
#url = "https://63gytpm.azure.babyrocket.net/invocations"
data_json = input_df.to_json(orient="split",index=False)
headers = {"Content-Type":"application/json; format=pandas-split"}
response = requests.post(url,data=data_json,headers=headers)
print(response.json())

[5, 4, 3, 3, 5, 5, 3, 5, 5, 5, 7, 5, 5, 5, 0, 5, 5, 5, 3, 9, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 9, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 7, 5, 5, 8, 5, 5, 5, 5, 5, 5, 5, 6, 8, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 0, 5, 5, 3, 5, 5, 2, 5, 5, 5, 5, 9, 5, 8, 5, 8, 3, 5, 5, 5, 5, 8, 1, 5, 3, 5, 5, 5, 3, 5, 6, 5]
